In [46]:
import urllib2
import urllib
from lxml import html
import re
list_o_movies = ['room', 'the-revenant-2015']

mc= 'http://www.metacritic.com/movie/room/critic-reviews'

In [47]:
url = mc
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
values = {'name' : 'Michael Foord',
          'location' : 'Northampton',
          'language' : 'Python' }
headers = { 'User-Agent' : user_agent }

data = urllib.urlencode(values)
req = urllib2.Request(url, data, headers)
response = urllib2.urlopen(req)
the_page = response.read()

In [48]:
tree = html.fromstring(the_page)

In [1]:
# tree.xpath('//a/text()')

In [16]:
#possibilities = re.findall('([^ ]+)="([^"/]*(?=article|body|story).*?[^/]?)"', tree.body,re.IGNORECASE)

In [17]:
path = '//li[@class="review_action full_review"]/a/@href'
links = tree.body.xpath(path)

In [18]:
import BeautifulSoup


In [26]:
r= 'http://www.imdb.com/find?ref_=nv_sr_fn&q=Room&s=all'
url = r
def get_tree(url):
    user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
    values = {'name' : 'Michael Foord',
	      'location' : 'Northampton',
	      'language' : 'Python' }
    headers = { 'User-Agent' : user_agent }

    data = urllib.urlencode(values)
    req = urllib2.Request(url, data, headers)
    response = urllib2.urlopen(req)
    the_page = response.read()
    tree = html.fromstring(the_page)	
    return tree

In [35]:
name = '//td[@class="result_text"]/a/text()'
path = '//td[@class="result_text"]/a/@href'
years = '//td[@class="result_text"]/text()'
full = '//td[@class="result_text"]/a'
movies = tree.body.xpath(full)


In [44]:
for a in movies:
    print a.attrib

{'href': '/title/tt3170832/?ref_=fn_al_tt_1'}
{'href': '/title/tt0417096/?ref_=fn_al_tt_2'}
{'href': '/title/tt0481321/?ref_=fn_al_tt_3'}
{'href': '/title/tt4062536/?ref_=fn_al_tt_4'}
{'href': '/title/tt0368226/?ref_=fn_al_tt_5'}
{'href': '/title/tt3952108/?ref_=fn_al_tt_6'}
{'href': '/title/tt2381111/?ref_=fn_al_tt_7'}
{'href': '/title/tt3482062/?ref_=fn_al_tt_8'}
{'href': '/title/tt3832914/?ref_=fn_al_tt_9'}
{'href': '/title/tt2467372/?ref_=fn_al_tt_10'}
{'href': '/name/nm2395937/?ref_=fn_al_nm_1'}
{'href': '/keyword/room/?ref_=fn_al_kw_1'}
{'href': '/company/co0538730/?ref_=fn_al_co_1'}
{'href': '/company/co0277535/?ref_=fn_al_co_2'}
{'href': '/company/co0482292/?ref_=fn_al_co_3'}
{'href': '/company/co0367303/?ref_=fn_al_co_4'}
{'href': '/company/co0482449/?ref_=fn_al_co_5'}


In [0]:
#get all year links 
#get 2016 results from first page
# get all results from each year
oscar_page = 'http://www.imdb.com/awards-central/oscars/'
